# CO2 analysis

This notebook is being used to prepare data and figures for the upcoming APERC Annual CO2 Emissions Report, due for publication in June 2020. The data and results here are strictly preliminary at this point.

A final interactive notebook will be made available at publication.

To use this notebook:
1. import the libraries below
2. import the prepared data
3. Plot all economies together
4. Plot each economy separately. You can select either fuels or sectors.

In [ ]:
import numpy as np
import pandas as pd
import glob
import os

#### Prepare data

In [ ]:
file_list = glob.glob('../data/CO2 raw/*-Co2r.xls')

In [ ]:
economies = []

for file in file_list:
    economies_dirty = os.path.basename(file)
    economies_dirty = economies_dirty[:-9]
    economies.append(economies_dirty)

In [ ]:
names = {
    'AUS':'01_AUS',
    'BRN':'02_BD',
    'CAN':'03_CDA',
    'CHL':'04_CHL',
    'CHN':'05_PRC',
    'HKG':'06_HKC',
    'IDN':'07_INA',
    'Jpn':'08_JPN',
    'KOR':'09_ROK',
    'MYS':'10_MAS',
    'MEX':'11_MEX',
    'NZL':'12_NZ',
    'PNG':'13_PNG',
    'PER':'14_PE',
    'PHL':'15_RP',
    'RUS':'16_RUS',
    'SGP':'17_SG',
    'TWN':'18_CT',
    'THA':'19_THA',
    'USA':'20_USA',
    'VNM':'21_VN'
}

In [ ]:
dfs = []
for file,economy in zip(file_list,economies):
    #print(file)
    _df = pd.read_excel(file,
                      sheet_name='kt-CO2',
                      skiprows=3,
                        nrows=19,
                       usecols='A:AM')
    _df = _df.rename({'Unnamed: 0':'Item'},axis=1)
   # _df = (_df.dropna(how="all",axis=1)
   #           .dropna(how="all", axis=0)
   #           .reset_index(drop=True)
   #           .rename({'Unnamed: 0':'Item'},axis=1))
    
    _df = _df.melt(id_vars=['Item'],
                var_name='Year',
                value_name='Value')
    _df['Economy'] = economy
    dfs.append(_df)
df = pd.concat(dfs)
df = df.replace(names)
df['Item'] = df['Item'].str.lstrip()
df['Item'] = df['Item'].str.rstrip('1')
df['Item'] = df['Item'].replace({'1.3  Gas':'1.3 Gas',
                                 '1.1  Coal & Coal Products':'1.1 Coal & Coal Products',
                                 '1.2  Crude Oil & Petroleum Products':'1.2 Crude Oil & Petroleum Products',
                                 '2.1.1 Public Utilities':'2.1.1 Main Activity Producer',
                                 '2.1.2 Auto-Generation':'2.1.2 Autoproducers',
                                 '2.1.4 Own Use & Loss':'2.1.4 Loss & Own Use',
                                 '1.4  Others':'1.4 Others'})
df['Value'] = np.divide(df['Value'],1000000)
df = df[df['Year'] >=1990]
economies = df['Economy'].unique().tolist()

df.to_csv('../data/CO2 emissions prepared.csv', index=False)

In [ ]:
df.head()